In [75]:
from wikidataintegrator import wdi_core, wdi_login, wdi_helpers
from wikidataintegrator.wdi_helpers import try_write
from getpass import getpass
import ipywidgets as widgets
from ipywidgets import TwoByTwoLayout
from IPython.display import IFrame, clear_output

## Login panel
### Label
wikidata = widgets.HTML(value="<H2>Login to Wikidata</H2>")

### Username
username_textbox = widgets.Text(
       value="",
        description="Username",
)
## Password
password_textbox = widgets.Password(description='Password:')
## Button
loginbutton = widgets.Button(description="login", icon='sign-in-alt')
## Login as
loginlabel = widgets.Label(value="Please login to Wikidata first")

@loginbutton.on_click
def wdlogin(b):
    login = wdi_login.WDLogin(username_textbox.value, password_textbox.value)
    creds = login.generate_edit_credentials()
    koekjes = creds.get_dict()
    loginlabel.value="Logged in to Wikidata as User: "+koekjes["wikidatawikiUserName"]
    b.login = login

 

file = open("GeneWikidata-logo-en.png", "rb")
image = file.read()
logo = widgets.Image(
    value=image,
    format='png',
    width=200,
)
    
loginPane = widgets.VBox(layout={'border': '1px solid black'},
    children=[wikidata, username_textbox, password_textbox, loginbutton, loginlabel])
header = TwoByTwoLayout(top_left=logo, 
               bottom_right=loginPane)   
widgets.HBox(children=[logo, loginPane])

### Tabs to identifier resolvers
#### Citations
doi_textbox = widgets.Text(
    value='',
    description='DOI',
)
     
wddoilabel = widgets.Label(value="")

doi_textbox = widgets.Text(
    value='',
    description='DOI',
)

progressbar = widgets.Output()

file = open("WikiciteWikidata.gif", "rb")
image = file.read()
progressgif = widgets.Image(
    value=image,
    format='gif',
    width=300,
    height=400,
)

# progressgif = IFrame(src="WikiciteWikidata", width="50", height="50")

DOIbutton = widgets.Button(
    description='Wikidata', icon="book-reader"
)
@DOIbutton.on_click
def DOIresolve(b):
    print(doi_textbox.value)
    with progressbar:
        clear_output()
        display(progressgif)
    doi_qid, _, _ = wdi_helpers.PublicationHelper(doi_textbox.value, id_type="doi",source="crossref").get_or_create(loginbutton.login)
    print(loginbutton.login)
    with progressbar:
        clear_output()

        ## Result tabs
        wikidatatab = IFrame(src='https://www.wikidata.org/wiki/'+doi_qid, width=1000, height=600)
        scholiatab = IFrame(src='https://scholia.toolforge.org/work/'+doi_qid, width=1000, height=600)
        tab1 = widgets.Output()
        tab2 = widgets.Output()
        tab = widgets.Tab(children=[
            tab1,
            tab2])
        with tab1:
            display(wikidatatab)
        with tab2:
            display(scholiatab)
        tab.set_title(0, 'Wikidata')
        tab.set_title(1, 'Scholia')
        display(tab)

doiComby = widgets.HBox(children=[doi_textbox, DOIbutton])
introduction = widgets.HTML("<h2>Resolve a DOI in Wikidata</h2>This webservice returns a Wikidata identifier or QID for a DOI. If the DOI is not covered in Wikidata an Item will be created, provided the DOI is recognised by crossref. (Example DOI: 10.1080/10665684.2011.613703 ")
form = widgets.VBox(children=[introduction, doiComby, progressbar]) # Citations
widgets.VBox(children=[header, form])

This service runs on WikidataIntegrator [![PyPi](https://img.shields.io/pypi/v/wikidataintegrator.svg)](https://pypi.python.org/pypi/wikidataintegrator) and integrates a DOI from <a href="https://www.crossref.org/" target="_new">Crossref</a> 